# This is a demo notebook of running Retriever-Reader Models for science multi-choice exam. 

## 1. **Retriever-Reader Models**

Retriever-Reader models, particularly the Contextualized Retrieval-Augmented Generation (RAG) approach, represent a breakthrough in Large Language Models (LLMs) such as GPT-3. These models integrate a dual-process system:

- **The Retriever**: This component is responsible for extracting pertinent documents or passages, such as from Wikipedia, to provide contextual grounding for the LLM's response generation. It's the cornerstone for sourcing external information that enhances the depth and accuracy of the responses.
  
- **The Reader**: Following retrieval, this component assimilates the gathered information to produce responses that are not only relevant but also informed by the additional context. This functionality is crucial in question-answering scenarios, particularly in educational settings where precision and reliability are key.

## 2. **The RAG Components**

### Search Engine Integration

- **Pyserini with Apache Lucene**: We employ Pyserini, a Python toolkit, for its seamless integration with Apache Lucene, a renowned Java-based search library. Lucene's advanced full-text indexing and search capabilities, combined with Pyserini's Python-friendly interface, form the backbone of our retrieval system.

### Ranking Criterion

- **BM25 Scoring**: Our system utilizes Lucene's BM25 scoring function for text retrieval. BM25 optimizes document ranking by considering term frequency and document length, providing a balanced approach to information retrieval. This method is crucial for selecting the most relevant passages to inform the LLM's responses.

## 3. **LLM Integration**

In this demo, we integrate a 70-billion-parameter Large Language Model using the Llama2 structure. This model choice is strategic for its advanced capabilities in understanding and generating human-like responses. To accommodate computational constraints, we employ a flexible sharding technique, ensuring optimal performance within Kaggle's GPU environment.

# Step 1: Setup libraries and Java

In [1]:
import os
!pip install -q /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --ignore-installed
# !pip install -q /kaggle/input/lucene-deps/pyserini-0.22.1-py3-none-any.whl --ignore-installed
!pip install -q pyserini
!git clone https://github.com/Robot-Eyes/flexible-LLM-sharding.git

Cloning into 'flexible-LLM-sharding'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 22), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (48/48), 261.95 KiB | 2.82 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [2]:
%env JAVA_HOME=/kaggle/input/lucene-deps/openjdk-20.0.2_linux-x64_bin/jdk-20.0.2/
!echo $JAVA_HOME
%env PATH=/kaggle/input/lucene-deps/openjdk-20.0.2_linux-x64_bin/jdk-20.0.2/bin:{os.environ['PATH']}
!echo $PATH

env: JAVA_HOME=/kaggle/input/lucene-deps/openjdk-20.0.2_linux-x64_bin/jdk-20.0.2/
/kaggle/input/lucene-deps/openjdk-20.0.2_linux-x64_bin/jdk-20.0.2/
env: PATH=/kaggle/input/lucene-deps/openjdk-20.0.2_linux-x64_bin/jdk-20.0.2/bin:/opt/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
/kaggle/input/lucene-deps/openjdk-20.0.2_linux-x64_bin/jdk-20.0.2/bin:/opt/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


# Step 2: RAG

In [3]:
import json
from typing import Any

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from pyserini.search.lucene import LuceneSearcher
from tqdm.auto import tqdm


def retrieval_row(
    searcher: LuceneSearcher,
    row: pd.Series,
    top_k: int,
    jsonl: bool,
    context_template: str,
    max_length: int,
    merge_method: str = "none",
    search_method: str = "normal",
) -> list[str]:
    if search_method == "normal":
        query = "{prompt} {A} {B} {C} {D} {E}".format(**row.to_dict())
        hits = searcher.search(q=query, k=top_k)

        retrieved_results: list[dict[str, Any]] = []
        for hit in hits:
            if jsonl:
                hit_dict = json.loads(hit.raw)
                hit_dict["contents"] = context_template.format_map(hit_dict)
            else:
                hit_dict = {"contents": hit.raw}
            retrieved_results.append(hit_dict)
    elif search_method in ["query_by_each_choice", "query_by_each_choice_kcut"]:
        retrieved_results_list = []
        row_dict = row.to_dict()
        prompt = row_dict["prompt"]
        for key in ["A", "B", "C", "D", "E"]:
            query = " ".join([prompt, row_dict[key]])
            hits = searcher.search(query, k=top_k)
            retrieved_results = []
            for _k in range(top_k):
                _res = json.loads(hits[_k].raw)
                _res["score"] = hits[_k].score
                retrieved_results.append(_res)
            retrieved_results_list.append(retrieved_results)

        retrieved_results_df = pd.DataFrame([item for sublist in retrieved_results_list for item in sublist])
        scores = retrieved_results_df["score"].values
        # Add rank score, which is much bigger than BM25 score.
        scores = scores.reshape(5, top_k) + (np.arange(top_k)[::-1] * 10000.0)
        scores = scores.reshape(5 * top_k)
        retrieved_results_df["rank_bm25_score"] = scores
        retrieved_results_df.drop_duplicates("id", keep="first", inplace=True)
        retrieved_results_df.sort_values("rank_bm25_score", ascending=False, inplace=True)
        retrieved_results = retrieved_results_df.to_dict("records")
        # Be careful that `retrieved_results` here can contain up to p.top_k * 5 candidates.
        if search_method == "query_by_each_choice_kcut":
            retrieved_results = retrieved_results[:top_k]
    else:
        raise ValueError(f"[ERROR] Unexpected value search_method={search_method}")

    if merge_method == "none":
        pass
    elif merge_method == "mergev2":
        merged_results = []
        contents = ""
        for _k, res in enumerate(retrieved_results):
            if len(contents) > 0 and len(contents) + len(res["contents"]) + 1 > max_length:
                merged_results.append({"contents": contents})
                contents = ""
                # print(f"[DEBUG] merged until {_k=}")
            if len(contents) > 0:
                contents += "\n"
            contents += res["contents"]
        if len(contents) > 0:
            merged_results.append({"contents": contents})
            # print(f"[DEBUG] merged until {len(retrieved_results)}")
        retrieved_results = merged_results
    else:
        raise ValueError(f"[ERROR] Unexpected value merge_method={merge_method}")

    _context_list: list[str] = [res["contents"][:max_length] for res in retrieved_results]
    return _context_list

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def run_lucene_retrieval(
    df,
    index_dir: str,
    jsonl: bool,
    max_length: int = 2000,
    top_k: int = 5,
    bm25_k1: float = 0.9,
    bm25_b: float = 0.4,
    n_jobs: int = 128,
    merge_method: str = "none",
    search_method: str = "normal",
):
    
    # initialize LuceneSearcher
    searcher = LuceneSearcher(index_dir)
    searcher.set_bm25(k1=bm25_k1, b=bm25_b)

    context_template = "{contents}"

    contexts: list[list[str]] = Parallel(n_jobs, backend="threading", verbose=1)(
        delayed(retrieval_row)(searcher, row, top_k, jsonl, context_template, max_length, merge_method, search_method)
        for _, row in tqdm(df.iterrows(), total=len(df))
    )
    context_df = pd.DataFrame(contexts)
        
    return context_df

In [5]:
df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv", index_col="id")

In [6]:
context_df = run_lucene_retrieval(df, index_dir='/kaggle/input/lucene-wikipedia-en/',
                                jsonl=True, max_length=2000, top_k=2, n_jobs=4)

for col_idx in range(context_df.shape[1]):
    df[f"context_{col_idx}"] = context_df.iloc[:, col_idx]

Nov 15, 2023 3:42:24 PM org.apache.lucene.store.MMapDirectory lookupProvider


  0%|          | 0/200 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   31.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:  1.4min finished


# Step 3: Run LLM

In [7]:
import os, sys, gc
from pathlib import Path
import pickle
import numpy as np
import pandas as pd

## Generate prompt pickle file

In [8]:
def prepare_input_prompts_with_context(df):

    instruction_str = ("Below is an instruction that describes a task, paired with an input that provides further context. "
                 "Write a response that appropriately completes the request.\n\n### "
                 "Instruction:\nYou are an expert in STEM fields. Your task is to analyze the question and the proposed statement below. "
                "If the statement is correct, respond True; if it is not correct, respond False. "
                   "As a potential aid for you, some background context is provide, and please note that they could be irrelevant to the question."
                 "\n### ### Context:\ncontext_text\nQuestion: ")
    
    prompt_prefix_list = pd.Series([instruction_str.replace('context_text', row.context_0+row.context_1) 
                                    for _,row in df.iterrows()]) + df['prompt'] + "\nProposed answer: "
    prompt_suffix_list = [tuple(f"{row[letter]}\n\n### Response:\n" for letter in "ABCDE")
                          for _,row in df.iterrows()]

    return list(zip(prompt_prefix_list, prompt_suffix_list))

input_prompts = prepare_input_prompts_with_context(df)
    
with open('input_prompts.pkl', 'wb') as file:
    pickle.dump(input_prompts, file)

### Create symlinks from kaggle datasets to fake cached model. This step could be skipped on local machines.

In [9]:
os.makedirs('./platypus2_checkpoints', exist_ok=True)
for part in [1, 2]:
    source_dir = Path(f"/kaggle/input/platypus2-70b-instruct-part{part}")
    for path in source_dir.glob("*"):
        try:
            (Path('./platypus2_checkpoints') / path.name).symlink_to(path)
        except:
            pass

In [10]:
!python flexible-LLM-sharding/main.py --layer_num_per_shard 8 --storage_location cpu --num_batch 1 \
                        --model_path ./platypus2_checkpoints \
                        --prompt_pickle input_prompts.pkl --output_file platypus2_output_score.pkl

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Namespace(model_path='./platypus2_checkpoints', prompt_pickle='input_prompts.pkl', output_file='platypus2_output_score.pkl', num_batch=1, layer_num_per_shard=8, storage_location='cpu', max_activation_in_cpu=100, data_parallel=False, disk_folder='./temp', num_gen_token=1)
cuda:0 | shards:   0%|                                    | 0/6 [00:00<?, ?it/s]
cuda:0 | shards:  17%|████▌                      | 1/6 [03:36<18:04, 216.88s/it]
cuda:0 | shards:  33%|█████████                  | 2/6 [07:50<15:52, 238.24s/it]
cuda:0 | shards:  50%|█████████████▌             | 3/6 [11:51<11:58, 239.58s/it]
cuda:0 | shards:  67%|██████████████████         | 4/6 [15:50<07:59, 239.60s/it]
cuda:0 | shards:  83%|██████████████████████▌    | 5/6 [19:57<04:02, 242.

In [11]:
with open('platypus2_output_score.pkl', 'rb') as file:
    output_scores = pickle.load(file)
        
n = len(output_scores)
for i, scores in enumerate(output_scores):
    # Token #5852 is true, #7700 is false.
    first_token_scores = scores[:, 0, [5852, 7700]]
    positive_scores = first_token_scores[:, 0] / first_token_scores.sum(axis=-1)
    top3 = np.argsort(positive_scores)[::-1][:3]
    df.loc[i, "prediction"] = " ".join(["ABCDE"[j] for j in top3])

# Display performances if train set is used
if "answer" in df.columns:
    df["top_1"] = df["prediction"].apply(lambda x:x[0])
    df["top_2"] = df["prediction"].apply(lambda x:x[2])
    df["top_3"] = df["prediction"].apply(lambda x:x[4])

    top_i = [(df[f"top_{i}"] == df["answer"]).sum() for i in [1, 2, 3]]
    print(f"top1 : {top_i[0]}/{n}, top2 : {top_i[1]}/{n}, top3 : {top_i[2]}/{n} (total={sum(top_i)} / {n})")
    print(f"Accuracy: {100*top_i[0]/n:.1f}%, map3: {100*(top_i[0] + top_i[1]*1/2 + top_i[2]*1/3).sum()/n:.1f}%")

top1 : 131/200, top2 : 33/200, top3 : 22/200 (total=186 / 200)
Accuracy: 65.5%, map3: 77.4%
